In [2]:
%matplotlib inline
import pandas as pd
import numpy as np
import pickle
import re

import matplotlib.pyplot as plt

Cluster master node: `ssh gaspar_account@iccluster040.iccluster.epfl.ch`\
Cluster log: http://iccluster040.iccluster.epfl.ch:8088/cluster\
```
hadoop fs -ls /datasets
hadoop fs -cat /datasets/<file_name> | less
```

#### Uploading Data

In [5]:
population = pd.read_csv(r'../data/population_data.csv')

In [36]:
global_population = population[['Year','Value']].groupby('Year').sum()
global_population.Value = global_population.Value*1000
global_population['population_billions'] = global_population.Value/1000000000
global_population[['population_billions']].tail()

,population_billions
Year,
2014,8.726229
2015,8.818261
2016,8.909819
2017,9.000759
2018,9.090746


Population  
First issue: how is the sum of the population of each country greater than the estimated global population of 7.7 billion...?? Maybe we should use this for individual countries population and find another source of global population?

In [24]:
# gleam info: http://www.fao.org/gleam/en/
gleam = pd.read_excel(r'../data/gleam_data.xlsx', sheet_name = 1, header = [0,1])
gleam.head(10)

,Unnamed: 0_level_0,Unnamed: 1_level_0,Unnamed: 2_level_0,Unnamed: 3_level_0,Emission intensity,Production,Total GHG emissions,Total CO2 emissions,Total CH4 emissions,Total N2O emissions,...,"Feed: fertilizer & crop residues, N2O","Feed: applied & deposited manure, N2O","LUC: soy & palm, CO2","LUC: pasture expansion, CO2","Enteric fermentation, CH4","Manure management, CH4","Manure management, N2O","Direct energy, CO2","Indirect energy, CO2","Postfarm, CO2"
,Region,Animal species,Production system,Commodity,kg CO2-eq · kg protein-1,kg protein,kg CO2-eq,kg CO2-eq,kg CO2-eq,kg CO2-eq,...,kg CO2-eq,kg CO2-eq,kg CO2-eq,kg CO2-eq,kg CO2-eq,kg CO2-eq,kg CO2-eq,kg CO2-eq,kg CO2-eq,kg CO2-eq
0,Global,Cattle,Aggregated,Aggregated,160.292608,2.916310e+10,4.674630e+12,9.449299e+11,2.648727e+12,1.080973e+12,...,1.740732e+11,7.419864e+11,4.687954e+10,3.870062e+11,2.509275e+12,1.394522e+11,1.649130e+11,4.215542e+10,1.585653e+10,9.955932e+10
1,Global,Cattle,Aggregated,Milk,86.728932,1.888089e+10,1.637519e+12,2.756914e+11,9.676442e+11,3.941834e+11,...,7.166208e+10,2.543885e+11,1.497421e+10,0.000000e+00,8.856352e+11,8.200906e+10,6.813278e+10,2.993485e+10,3.252889e+09,8.929544e+10
2,Global,Cattle,Aggregated,Meat,295.375060,1.028222e+10,3.037111e+12,6.692385e+11,1.681083e+12,6.867892e+11,...,1.024111e+11,4.875979e+11,3.190534e+10,3.870062e+11,1.623640e+12,5.744310e+10,9.678019e+10,1.222057e+10,1.260364e+10,1.026388e+10
3,Global,Cattle,Grassland systems,Aggregated,206.327939,1.033817e+10,2.133054e+12,5.628062e+11,1.052965e+12,5.172829e+11,...,5.253660e+10,4.267612e+11,8.194330e+09,3.870062e+11,9.969096e+11,5.605552e+10,3.798508e+10,1.417326e+10,4.303272e+09,3.773092e+10
4,Global,Cattle,Grassland systems,Milk,94.988798,6.940654e+09,6.592844e+11,9.286140e+10,3.713461e+11,1.950770e+11,...,2.156126e+10,1.587838e+11,2.412100e+09,0.000000e+00,3.369018e+11,3.444428e+10,1.473188e+10,1.078605e+10,9.998435e+08,3.462338e+10
5,Global,Cattle,Grassland systems,Meat,433.778096,3.397520e+09,1.473770e+12,4.699448e+11,6.816191e+11,3.222059e+11,...,3.097534e+10,2.679774e+11,5.782229e+09,3.870062e+11,6.600079e+11,2.161124e+10,2.325320e+10,3.387219e+09,3.303428e+09,3.107542e+09
6,Global,Cattle,Mixed systems,Aggregated,138.681662,1.730617e+10,2.400048e+12,3.529705e+11,1.522347e+12,5.247301e+11,...,1.108839e+11,2.969106e+11,3.661036e+10,0.000000e+00,1.443988e+12,7.835867e+10,1.169357e+11,2.481956e+10,9.048642e+09,5.896378e+10
7,Global,Cattle,Mixed systems,Milk,81.927612,1.194023e+10,9.782346e+11,1.828300e+11,5.962982e+11,1.991064e+11,...,5.010081e+10,9.560470e+10,1.256211e+10,0.000000e+00,5.487334e+11,4.756479e+10,5.340090e+10,1.914881e+10,2.253045e+09,5.467206e+10
8,Global,Cattle,Mixed systems,Meat,264.970279,5.365935e+09,1.421813e+12,1.701405e+11,9.260490e+11,3.256237e+11,...,6.078305e+10,2.013059e+11,2.404825e+10,0.000000e+00,8.952551e+11,3.079388e+10,6.353482e+10,5.670750e+09,6.795597e+09,4.291724e+09


In [32]:
emissions = pd.read_csv(r'../data/emissions_data.csv', encoding = 'iso-8859-1')
# TODO rename columns without the Y
emissions.head()

,Area Code,Area,Item Code,Item,Element Code,Element,Unit,Y1961,Y1962,Y1963,...,Y2010,Y2011,Y2012,Y2013,Y2014,Y2015,Y2016,Y2017,Y2030,Y2050
0,2,Afghanistan,5058,Enteric Fermentation,7225,Emissions (CH4),gigagrams,240.6831,245.3106,255.8285,...,401.0680,402.5130,396.9210,393.0930,398.2870,384.1370,381.6789,371.9253,453.7474,603.6185
1,2,Afghanistan,5058,Enteric Fermentation,7231,Emissions (CO2eq),gigagrams,5054.3459,5151.5228,5372.3989,...,8422.4280,8452.7730,8335.3410,8254.9530,8364.0270,8066.8761,8015.2578,7810.4313,9528.6962,12675.9880
2,2,Afghanistan,5058,Enteric Fermentation,7244,Emissions (CO2eq) from CH4,gigagrams,5054.3459,5151.5228,5372.3989,...,8422.4280,8452.7730,8335.3410,8254.9530,8364.0270,8066.8761,8015.2578,7810.4313,9528.6962,12675.9880
3,2,Afghanistan,5059,Manure Management,7225,Emissions (CH4),gigagrams,11.6228,11.9632,12.6064,...,26.5669,26.1599,26.0913,26.0859,26.3682,24.9017,24.7679,23.7619,27.2114,35.2700
4,2,Afghanistan,5059,Manure Management,7231,Emissions (CO2eq),gigagrams,367.8310,376.4362,392.5811,...,681.2759,678.8488,672.3300,667.6991,675.6191,645.1936,641.9611,623.3735,750.3240,1003.2279


Emissions  
Note: There are countries and geographical areas included under the 'Area' heading, so we can't do any sums on the whole dataframe

In [35]:
supply = pd.read_csv(r'../data/supply_data.csv', encoding = 'iso-8859-1')
supply.head()

# TODO need to break this down to get the data we actually want to use, eg just one unit, sorted by year, certain regions...
# Need to work out what supply actually refers to? Is it the country's production or consumption of these products?


,Area Code,Area,Item Code,Item,Element Code,Element,Unit,Y1961,Y1961F,Y1962,...,Y2009,Y2009F,Y2010,Y2010F,Y2011,Y2011F,Y2012,Y2012F,Y2013,Y2013F
0,2,Afghanistan,2731,Bovine Meat,641,Food supply quantity (tonnes),tonnes,43000.00,S,45800.00,...,136279.00,S,133397.00,S,140660.00,S,153438.00,S,140087.00,S
1,2,Afghanistan,2731,Bovine Meat,646,Food supply quantity (g/capita/day),g/capita/day,13.16,Fc,13.73,...,13.48,Fc,12.87,Fc,13.24,Fc,14.09,Fc,12.56,Fc
2,2,Afghanistan,2731,Bovine Meat,645,Food supply quantity (kg/capita/yr),kg,4.80,Fc,5.01,...,4.92,Fc,4.70,Fc,4.83,Fc,5.14,Fc,4.59,Fc
3,2,Afghanistan,2731,Bovine Meat,664,Food supply (kcal/capita/day),kcal/capita/day,28.00,Fc,30.00,...,29.00,Fc,28.00,Fc,28.00,Fc,30.00,Fc,27.00,Fc
4,2,Afghanistan,2731,Bovine Meat,674,Protein supply quantity (g/capita/day),g/capita/day,1.99,Fc,2.07,...,2.03,Fc,1.94,Fc,2.00,Fc,2.13,Fc,1.89,Fc
